In [27]:
## TRANSFORMATION 
import pandas as pd

In [28]:
combined_patients = pd.read_csv("../../datasets/combined_patients.csv")
combined_transactions = pd.read_csv("../../datasets/combined_transactions.csv")

In [29]:
# Remove Duplicate Records
print("Before : ",combined_patients.shape[0]," rows")
patient_df = combined_patients.drop_duplicates()
print("After : ",patient_df.shape[0], "rows")


print("Before : ",combined_transactions.shape[0]," rows")
transaction_df = combined_transactions.drop_duplicates()
print("After : ",transaction_df.shape[0], "rows")

Before :  9985  rows
After :  9985 rows
Before :  20000  rows
After :  20000 rows


In [31]:
# standardize name format

for col in ["FirstName", "MiddleName", "LastName"]:
    if col in patient_df.columns:
        patient_df[col] = patient_df[col].str.title()
patient_df.head(2)

,PatientID,FirstName,LastName,MiddleName,SSN,PhoneNumber,Gender,DOB,Address,ModifiedDate,source,unified_patient_id
0,HOSP1-000001,Rick,Russo,U,188-23-9828,+1-630-829-7585x0769,Female,1937-06-04,"Unit 0915 Box 7064, DPO AA 82777",2020-05-25,hospital_a,HOSP_A_HOSP1-000001
1,HOSP1-000002,Gregory,Graham,B,730-45-8217,456.746.7289x69233,Female,1937-06-10,"9864 Gibson Islands, Danielside, KY 99809",2021-06-05,hospital_a,HOSP_A_HOSP1-000002


In [33]:
patient_df.columns

Index(['PatientID', 'FirstName', 'LastName', 'MiddleName', 'SSN',
       'PhoneNumber', 'Gender', 'DOB', 'Address', 'ModifiedDate', 'source',
       'unified_patient_id'],
      dtype='object')

In [32]:
# Clean & Format Phone Numbers

import re

def clean_phone_number(number):
    # Remove extension if present
    number = re.sub(r'x\d+', '', number)
    
    # Remove non-digit characters
    digits = re.sub(r'\D', '', number)
    
    # Handle leading country code (assuming US, 1)
    if digits.startswith("001"):
        digits = digits[3:]
    elif digits.startswith("1") and len(digits) > 10:
        digits = digits[1:]
    
    # Format as (XXX) XXX-XXXX if 10 digits
    if len(digits) == 10:
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
    else:
        return None  # Or return digits if you want to keep it

# Example usage
patient_df["PhoneNumber"] = patient_df["PhoneNumber"].apply(clean_phone_number)


In [34]:
patient_df["PhoneNumber"]


0       (630) 829-7585
1       (456) 746-7289
2       (522) 501-5461
3       (345) 608-9409
4       (963) 994-2969
             ...      
9980    (752) 327-3814
9981    (987) 393-8874
9982    (648) 293-8022
9983    (202) 943-9539
9984    (316) 603-3639
Name: PhoneNumber, Length: 9985, dtype: object

In [35]:
patient_df.head(2)

,PatientID,FirstName,LastName,MiddleName,SSN,PhoneNumber,Gender,DOB,Address,ModifiedDate,source,unified_patient_id
0,HOSP1-000001,Rick,Russo,U,188-23-9828,(630) 829-7585,Female,1937-06-04,"Unit 0915 Box 7064, DPO AA 82777",2020-05-25,hospital_a,HOSP_A_HOSP1-000001
1,HOSP1-000002,Gregory,Graham,B,730-45-8217,(456) 746-7289,Female,1937-06-10,"9864 Gibson Islands, Danielside, KY 99809",2021-06-05,hospital_a,HOSP_A_HOSP1-000002


In [38]:
# validate email address

email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'

def is_valid_email(email):
    return bool(re.match(email_pattern, str(email))) if pd.notnull(email) else False

if "Email" in patient_df.columns:
    patient_df["Email"] = patient_df["Email"].apply(is_valid_email)
else:
    print("Email column is not found")

Email column is not found


In [40]:
# Compute DQ flag
patient_df["dq_flag"] = (
    patient_df["PhoneNumber"].isnull() |
    patient_df["DOB"].isnull() |
    (patient_df["Gender"].isnull()) 
)

In [43]:
patient_df["dq_flag"]==True

0       False
1       False
2       False
3       False
4       False
        ...  
9980    False
9981    False
9982    False
9983    False
9984    False
Name: dq_flag, Length: 9985, dtype: bool

In [44]:
##  Task 3.2: Business Logic Implementation

In [ ]:
# Calculate Patient Age from DOB


from datetime import datetime

today = pd.to_datetime("today")
patient_df["DOB"] = pd.to_datetime(patient_df["DOB"], errors="coerce")
patient_df["age"] = patient_df["DOB"].apply(lambda dob: int((today - dob).days / 365.25) if pd.notnull(dob) else None)



In [48]:
patient_df["age"]

0       88
1       88
2       98
3       53
4       97
        ..
9980    90
9981    57
9982    28
9983    78
9984    48
Name: age, Length: 9985, dtype: int64

In [49]:
# Compute Insurance Coverage Percentages

transaction_df["Amount"] = pd.to_numeric(transaction_df["Amount"], errors="coerce")
transaction_df["PaidAmount"] = pd.to_numeric(transaction_df["PaidAmount"], errors="coerce")

transaction_df["coverage_pct"] = (
    transaction_df["PaidAmount"] / transaction_df["Amount"]
).round(2)


In [50]:
transaction_df["coverage_pct"]

0        0.32
1        2.29
2        6.51
3        0.55
4        0.42
         ... 
19995    0.08
19996    0.21
19997    0.33
19998    0.52
19999    0.25
Name: coverage_pct, Length: 20000, dtype: float64

In [51]:
# Categorize Payment Statuses

def get_payment_status(row):
    if pd.isna(row["PaidAmount"]) or row["PaidAmount"] == 0:
        return "Denied"
    elif row["PaidAmount"] >= row["Amount"]:
        return "Paid"
    elif row["PaidAmount"] > 0 and row["PaidAmount"] < row["Amount"]:
        return "Partial"
    else:
        return "Pending"

transaction_df["payment_status"] = transaction_df.apply(get_payment_status, axis=1)


In [52]:
transaction_df["payment_status"]

0        Partial
1           Paid
2           Paid
3        Partial
4        Partial
          ...   
19995    Partial
19996    Partial
19997    Partial
19998    Partial
19999    Partial
Name: payment_status, Length: 20000, dtype: object

In [53]:
# Add Time Dimensions from ServiceDate

transaction_df["ServiceDate"] = pd.to_datetime(transaction_df["ServiceDate"], errors="coerce")

transaction_df["year"] = transaction_df["ServiceDate"].dt.year
transaction_df["month"] = transaction_df["ServiceDate"].dt.month
transaction_df["quarter"] = transaction_df["ServiceDate"].dt.quarter
transaction_df["day_of_week"] = transaction_df["ServiceDate"].dt.day_name()



In [57]:
transaction_df["month"]

0        5
1        9
2        3
3        2
4        4
        ..
19995    3
19996    5
19997    3
19998    1
19999    3
Name: month, Length: 20000, dtype: int32

In [ ]:
#Task 3.3: Common Data Model (CDM)

In [58]:

patient_df.columns


Index(['PatientID', 'FirstName', 'LastName', 'MiddleName', 'SSN',
       'PhoneNumber', 'Gender', 'DOB', 'Address', 'ModifiedDate', 'source',
       'unified_patient_id', 'dq_flag', 'age'],
      dtype='object')

In [59]:
# Standardize Procedure Codes and Descriptions

transaction_df["ProcedureCode"] = transaction_df["ProcedureCode"].astype(str).str.upper().str.strip()


In [60]:
transaction_df["ProcedureCode"]

0        94521
1        51588
2        32053
3        21422
4        39210
         ...  
19995    31940
19996    41380
19997    43643
19998    83762
19999    99441
Name: ProcedureCode, Length: 20000, dtype: object